## ● 網絡分析 Ming Hsun
## 資料來源:具有經緯度座標之Flickr打卡資料(2008-2014)
## 目地:建立台南地區鄉鎮單元間 旅遊地的網絡分析圖

In [3]:
import shapefile
import pyproj
import pygmaps
import math
import time
import cPickle as pickle
import json

### 讀取geodatabase 取得圖層

In [4]:
# standard imports
import sys
# import OGR
from osgeo import ogr
# use OGR specific exceptions
ogr.UseExceptions()
# get the driver
driver = ogr.GetDriverByName("OpenFileGDB")
gdb = driver.Open(r"F:\Output.gdb", 0)
gdb
layer = gdb.GetLayer("tainan_town")
# list to store layers'names
featsClassList = []
# parsing layers by index
for featsClass_idx in range(gdb.GetLayerCount()):
    featsClass = gdb.GetLayerByIndex(featsClass_idx)
    featsClassList.append(featsClass.GetName())
layerDefinition = layer.GetLayerDefn()

### 了解圖層中欄位名稱

In [6]:
for i in range(layerDefinition.GetFieldCount()):
    print layerDefinition.GetFieldDefn(i).GetName()

OBJECTID
PhotoId
UserId
OwnerName
Title
Latitude
Longitude
DateTaken
DateUpluad
Descriptio
Tags
URL
FID_1_1
countyid
countyname
0M�
;�
[��
townid
towncode
countyna_1
townname
node


### 本次project所需欄位:
### [UserId:使用者ID Latitude:緯度 Longitude:經度 DateTaken:拍攝時間 node:依據鄉鎮名稱進行編碼之號碼]
### 原始資料範例:"UserID:31559750@N02"  "Latitude:24.74588" "Longtitude:120.899911" "DataTaken:2014/7/6 下午 06:33:46" "node:12"
#### node:('七股':0,'下營':1,'大內':2,'山上':3,'中西':4,'仁德':5,'六甲':6,'北門':7,'左鎮':8,'永康':9,'玉井':10,'安平':11,'白河':12,'安定':13,'安南':14,'西港':15,'佳里':16,'官田':17,'東區':18,'南化':19,'南區':20,'後壁':21,'柳營':22,'將軍':23,'麻豆':24,'善化':25,'新化':26,'新市':27,'新營':28,'楠西':29,'學甲':30,'龍崎':31,'歸仁':32,'關廟':33,'鹽水':34,'北區':35,'東山':36)

##*讀取資料並進行資料前處理:
####1.將經緯度座標 轉換為 可計算之投影座標
####2.將時間轉換為可計算之timestamp

In [13]:
#prepare for the data 
prj = pyproj.Proj(init="epsg:3828")##定義投影橢求體
point_dict={}#checkdict
point_list=[]#userID_list
date_convert=[]
pid_list=[]
for feature in layer:
    try:
          if isinstance(feature.GetField("UserId"), str):#確定User ID 為文字
            split_=feature.GetField("DateTaken").split(' ')  #將2014/2/19 上午 08:36:23依據 空白切割
            split_D=split_[0].split('/') #將2014/2/19 依據'/'切割
            split_T=split_[2].split(':') #將03:00:49 依據':'切割
            split_D[1]=split_D[1].zfill(2)
            split_D[2]=split_D[2].zfill(2)
            D_s=split_D[0]+split_D[1]+split_D[2]
            T_s=split_T[0]+split_T[1]+split_T[2]
            x="{:.7f}".format(float(feature.GetField("Longitude")))
            y="{:.7f}".format(float(feature.GetField("Latitude")))
            photoPx, photoPy = prj(x,y, inverse=False)
            ph_x="{:.7f}".format(photoPx)
            ph_y="{:.7f}".format(photoPy)
            ph_ID=feature.GetField("PhotoId")
            if split_[1]=='\xe4\xb8\x8b\xe5\x8d\x88': #如果是下午
                split_T[0]=str(int(split_T[0])+12)  #如果是下午時就加上12小時 
                #date_convert=str(feature.GetField("UserId"))+split_D[0]+split_D[1]+split_D[2]+split_T[0]+split_T[1]+split_T[2]+','+str(ph_x)+str(ph_y)
            time_tuple=(int(split_D[0]), int(split_D[1]), int(split_D[2]), int(split_T[0]), int(split_T[1]),int(split_T[2]), 0, 0, 0)
            timestamp=time.mktime(time_tuple)    
            #checklist=[timestamp,date_convert,D_s,T_s,x,y,ph_x,ph_y,ph_ID]
#             checklist=[timestamp,D_s,T_s,x,y,ph_x,ph_y,ph_ID]
            town_id=feature.GetField("node")
            checklist=[timestamp,D_s,T_s,x,y,ph_x,ph_y,town_id]
            pid_list.append(ph_ID)
    #       print checklist
            #print photoPx,photoPy
            if feature.GetField("UserId") not in point_dict:
                point_list.append([(feature.GetField("UserId")),checklist])
                point_dict.setdefault(feature.GetField("UserId"),[checklist])
            else:
                point_list.append([(feature.GetField("UserId")),checklist])
                point_dict[feature.GetField("UserId")].append(checklist)
    except:
        continue


In [17]:
print "資料總數:",len(point_list),"筆"

資料總數: 594967 筆


### *處理後資料 格式:
#### ["使用者id",["可計算之時間","原始日期","原始時間","經度","緯度","投影後經度","投影後緯度","所在鄉鎮編碼"]]

In [18]:
for aa in range(0,5):
    print  point_list[aa]

['102069006@N07', [1419068598.0, '20141220', '054318', '120.0673970', '23.0422440', '154423.2197554', '2549274.2250152', 14]]
['61038718@N03', [1356166451.0, '20121222', '045411', '120.0962730', '23.0290290', '157373.6852379', '2547792.1858338', 14]]
['61038718@N03', [1352621620.0, '20121111', '041340', '120.0998560', '23.0271130', '157739.6371634', '2547577.7399433', 14]]
['92326125@N00', [1242091269.0, '20090512', '092109', '120.0694500', '23.0453350', '154635.8084898', '2549615.1959529', 14]]
['102069006@N07', [1420277244.0, '20150103', '052724', '120.0674550', '23.0420300', '154429.0131967', '2549250.4878923', 14]]


In [19]:
#處理重複的點資料 (同時且同地視為一筆資料)
unique=[]
for dupli in point_list:
    if dupli not in unique:
        unique.append(dupli)
print '同user同時同地的重複資料去除後 總數:',len(unique)

同user同時同地的重複資料去除後 總數: 88921


In [23]:
#由於資料數量較多 資料存成pickle 方便後續使用
# import cPickle as pickle
# print len(unique)
# print unique[1]
# pickle.dump(unique, open('tainan_town.p', 'w'))

88921
['61038718@N03', [1356166451.0, '20121222', '045411', '120.0962730', '23.0290290', '157373.6852379', '2547792.1858338', 14]]


### load pickle

## *建立使用路徑dictionary
### key:使用者id
### value:使用者之所有打卡點位(並且依據時間先後進行排序)

In [20]:
Check_dict1={}
for aa in unique:
        if aa[0] not in Check_dict1:
            Check_dict1.setdefault(aa[0],[aa[1]])
        else:
              Check_dict1[aa[0]].append(aa[1])
#depending on the time for soring
for sort in Check_dict1:
    Check_dict1[sort].sort()

In [21]:
print "使用者總數:",len(Check_dict1)

使用者總數: 2925


## *繪製網絡圖  link&node

#### 依據格式:
##### {"nodes": [{"group": 1, "name": "鄉鎮名"},...], "links": [{"source": 0, "target": 1, "value": 21}, ...]}

### 每一個使用者各打卡點所形成的為打卡者之路徑
### 本次為繪製網絡圖 因而將每個使用者打卡位置由時間前後 依據所在鄉鎮 化分成為pair
## pair=[出發鄉鎮(from),抵達鄉鎮(to)]

In [55]:
#打成pair
pair=[]
for user in Check_dict1:
    f=0
    t=1
    for point in range(1,len(Check_dict1[user])):
        pair.append([Check_dict1[user][f][-1],Check_dict1[user][t][-1]])
        f=f+1
        t=t+1
print "pair example:"
for p in range(0,20):
    print pair[p]

pair example:
[5, 4]
[4, 5]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[25, 25]
[32, 32]
[32, 32]
[32, 32]
[20, 4]
[4, 4]
[4, 20]
[20, 20]


In [45]:
print "總pair數:",len(pair)

總pair數: 85996


### 然而透過觀察 了解有需多的pair 位在同一個鄉鎮之內並沒有跨鄉鎮
### 為繪製各鄉鎮的網絡圖 因此只留下跨不同鄉鎮的 pair數

In [46]:
##移除位於同一鄉鎮中的pair
pair_list=[]
for net in pair:
    if not net[0]==net[1]:
        pair_list.append(net)

In [47]:
print "跨鄉鎮pair總數:",len(pair_list)
print "ex:",pair_list[0],pair_list[1],pair_list[2],pair_list[3]

跨鄉鎮pair總數: 7307
ex: [5, 4] [4, 5] [20, 4] [4, 20]


## *完成link
#### "links": [{"source": 0, "target": 1, "value": 21}, ...]}

In [38]:
link_dict={}
a=1
for l in pair_list:
    if str(l) not in link_dict:
        link_dict.setdefault(str(l),[l,1])
    else:
        link_dict[str(l)][-1]=link_dict[str(l)][-1]+1

In [39]:
links=[]
for net in link_dict:
    links.append({"source":link_dict[net][0][0], "target":link_dict[net][0][1], "value":link_dict[net][1]})

In [59]:
print "links example:"
for cc in range(0,15):
    print links[cc]

links example:
{'source': 20, 'target': 6, 'value': 1}
{'source': 24, 'target': 14, 'value': 5}
{'source': 0, 'target': 36, 'value': 4}
{'source': 24, 'target': 11, 'value': 7}
{'source': 33, 'target': 34, 'value': 1}
{'source': 5, 'target': 20, 'value': 10}
{'source': 25, 'target': 35, 'value': 5}
{'source': 24, 'target': 26, 'value': 4}
{'source': 13, 'target': 17, 'value': 2}
{'source': 34, 'target': 35, 'value': 2}
{'source': 16, 'target': 30, 'value': 10}
{'source': 9, 'target': 21, 'value': 7}
{'source': 14, 'target': 5, 'value': 22}
{'source': 8, 'target': 9, 'value': 4}
{'source': 9, 'target': 35, 'value': 58}


## *完成nodes
#### {"nodes": [{"group": 1, "name": "鄉鎮名"},...]

In [48]:
#自行定義之鄉鎮node編碼
town={'七股':0,'下營':1,'大內':2,'山上':3,'中西':4,'仁德':5,'六甲':6,'北門':7,'左鎮':8,'永康':9,'玉井':10,'安平':11,'白河':12,'安定':13
,'安南':14,'西港':15,'佳里':16,'官田':17,'東區':18,'南化':19,'南區':20,'後壁':21,'柳營':22,'將軍':23,'麻豆':24,'善化':25,'新化':26,
'新市':27,'新營':28,'楠西':29,'學甲':30,'龍崎':31,'歸仁':32,'關廟':33,'鹽水':34,'北區':35,'東山':36}


In [49]:
#完成node
nodes=[]
for t_name in  town :
    nodes.append({'group':town[t_name], "name": t_name})

In [51]:
print "鄉鎮(node)總數:",len(nodes)
print nodes

鄉鎮(node)總數: 37
[{'group': 2, 'name': '\xe5\xa4\xa7\xe5\x85\xa7'}, {'group': 20, 'name': '\xe5\x8d\x97\xe5\x8d\x80'}, {'group': 9, 'name': '\xe6\xb0\xb8\xe5\xba\xb7'}, {'group': 27, 'name': '\xe6\x96\xb0\xe5\xb8\x82'}, {'group': 21, 'name': '\xe5\xbe\x8c\xe5\xa3\x81'}, {'group': 13, 'name': '\xe5\xae\x89\xe5\xae\x9a'}, {'group': 3, 'name': '\xe5\xb1\xb1\xe4\xb8\x8a'}, {'group': 12, 'name': '\xe7\x99\xbd\xe6\xb2\xb3'}, {'group': 7, 'name': '\xe5\x8c\x97\xe9\x96\x80'}, {'group': 25, 'name': '\xe5\x96\x84\xe5\x8c\x96'}, {'group': 26, 'name': '\xe6\x96\xb0\xe5\x8c\x96'}, {'group': 19, 'name': '\xe5\x8d\x97\xe5\x8c\x96'}, {'group': 15, 'name': '\xe8\xa5\xbf\xe6\xb8\xaf'}, {'group': 0, 'name': '\xe4\xb8\x83\xe8\x82\xa1'}, {'group': 34, 'name': '\xe9\xb9\xbd\xe6\xb0\xb4'}, {'group': 24, 'name': '\xe9\xba\xbb\xe8\xb1\x86'}, {'group': 17, 'name': '\xe5\xae\x98\xe7\x94\xb0'}, {'group': 32, 'name': '\xe6\xad\xb8\xe4\xbb\x81'}, {'group': 1, 'name': '\xe4\xb8\x8b\xe7\x87\x9f'}, {'group': 31, 'name':

# *link&node匯出形成json檔

In [135]:
jsondata = {"nodes": nodes, "links": links}
json.dump(jsondata, open('tainan_town_network.json', 'w'))

In [ ]:
##引入D3 繪製HTML結果 (可複製url看D3視覺化成果)
http://www.minghsun.com/t_network_tainan.html

In [2]:
from IPython.display import HTML
HTML('<iframe src=http://www.minghsun.com/t_network_tainan.html width=1000 height=1000></iframe>')